In [54]:
from glmsingle.glmsingle import GLM_single
import argparse
import os
import os.path as op
from nilearn import image
from numrisk.utils.data import Subject
from nilearn.glm.first_level import make_first_level_design_matrix
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

bids_folder = '/Volumes/mrenkeED/data/ds-dnumr'

In [55]:
subject = '01'
session = 1
derivatives = op.join(bids_folder, 'derivatives')

runs = range(1, 7)
sub = Subject(subject, bids_folder=bids_folder)

ims = sub.get_preprocessed_bold(session=session)

base_dir = 'glm_stim1.denoise'

In [56]:
data = [image.load_img(im).get_fdata() for im in ims]

base_dir = op.join(derivatives, base_dir, f'sub-{subject}',
                    f'ses-{session}', 'func')



In [38]:
np.shape(image.load_img(ims[0]).get_fdata())[3]

188

In [57]:
onsets = sub.get_fmri_events(session=session, runs = runs)
print(np.shape(onsets))
onsets.head()

tr = 2.3
n = np.shape(image.load_img(ims[0]).get_fdata())[3]  # before: 135  np.shape(data)[2]
frametimes = np.linspace(tr/2., (n - .5)*tr, n)
onsets['onset'] = ((onsets['onset']+tr/2.) // 2.3) * 2.3

(360, 5)


In [58]:
onsets.loc[1].sort_values('onset')

,onset,trial_nr,trial_type,duration,n2
run,,,,,
1,6.9,1,trial_001_n1,0.6,NaN
1,13.8,1,n2_2,0.6,2.0
1,20.7,2,trial_002_n1,0.6,NaN
1,25.3,2,n2_28,0.6,28.0
1,32.2,3,trial_003_n1,0.6,NaN
1,39.1,3,n2_5,0.6,5.0
1,46.0,4,trial_004_n1,0.6,NaN
1,55.2,4,n2_28,0.6,28.0
1,59.8,5,trial_005_n1,0.6,NaN


In [41]:

dm = [make_first_level_design_matrix(frametimes, onsets.loc[run], hrf_model='fir', oversampling=100.,
                                        drift_order=0,
                                        drift_model=None).drop('constant', axis=1) for run in runs]
dm = pd.concat(dm, keys=range(1, 7), names=['run']).fillna(0)
dm.columns = [c.replace('_delay_0', '') for c in dm.columns]
dm /= dm.max()
#print(dm)
dm[dm < 1.0] = 0.0
print(dm.shape)

(1128, 202)


In [42]:
X = [dm.loc[run].values for run in runs]

opt = dict()
opt['wantlibrary'] = 1
opt['wantglmdenoise'] = 1
opt['wantfracridge'] = 1
opt['wantfileoutputs'] = [0, 0, 0, 1]


In [34]:
glmsingle_obj = GLM_single(opt)

results_glmsingle = glmsingle_obj.fit(
    X,
    data,
    0.6,
    2.3,
    outputdir=base_dir)

betas = results_glmsingle['typed']['betasmd']
betas = image.new_img_like(ims[0], betas)
betas = image.index_img(betas, slice(None, None, 2))







*** FITTING TYPE-A MODEL (ONOFF) ***

fitting model...
done.

preparing output...
done.

computing model fits...
done.

computing R^2...
done.

computing SNR...
done.

*** Setting brain R2 threshold to 0.13754772333608822 ***

*** FITTING TYPE-B MODEL (FITHRF) ***



chunks:   0%|          | 0/5 [00:00<?, ?it/s]

One or more regressors are all zeros. 
we will estimate a 0 weight for those regressors.
One or more regressors are all zeros. 
we will estimate a 0 weight for those regressors.
One or more regressors are all zeros. 
we will estimate a 0 weight for those regressors.
One or more regressors are all zeros. 
we will estimate a 0 weight for those regressors.
One or more regressors are all zeros. 
we will estimate a 0 weight for those regressors.
One or more regressors are all zeros. 
we will estimate a 0 weight for those regressors.
One or more regressors are all zeros. 
we will estimate a 0 weight for those regressors.
One or more regressors are all zeros. 
we will estimate a 0 weight for those regressors.
One or more regressors are all zeros. 
we will estimate a 0 weight for those regressors.
One or more regressors are all zeros. 
we will estimate a 0 weight for those regressors.
One or more regressors are all zeros. 
we will estimate a 0 weight for those regressors.
One or more regressor

chunks:  20%|██        | 1/5 [00:23<01:33, 23.47s/it]

One or more regressors are all zeros. 
we will estimate a 0 weight for those regressors.
One or more regressors are all zeros. 
we will estimate a 0 weight for those regressors.
One or more regressors are all zeros. 
we will estimate a 0 weight for those regressors.
One or more regressors are all zeros. 
we will estimate a 0 weight for those regressors.
One or more regressors are all zeros. 
we will estimate a 0 weight for those regressors.
One or more regressors are all zeros. 
we will estimate a 0 weight for those regressors.
One or more regressors are all zeros. 
we will estimate a 0 weight for those regressors.
One or more regressors are all zeros. 
we will estimate a 0 weight for those regressors.
One or more regressors are all zeros. 
we will estimate a 0 weight for those regressors.
One or more regressors are all zeros. 
we will estimate a 0 weight for those regressors.
One or more regressors are all zeros. 
we will estimate a 0 weight for those regressors.
One or more regressor

chunks:  20%|██        | 1/5 [00:41<02:45, 41.40s/it]

One or more regressors are all zeros. 
we will estimate a 0 weight for those regressors.


KeyboardInterrupt: 

In [43]:
np.shape(X)

(6, 188, 202)

In [44]:
print(dm.shape)
6 * 135

(1128, 202)


810

In [45]:
np.shape(data)

(6, 59, 71, 49, 188)

In [18]:
ims

['/Volumes/mrenkeED/data/ds-dnumr/derivatives/fmriprep/sub-01/ses-1/func/sub-01_ses-1_task-magjudge_run-1_space-T1w_desc-preproc_bold.nii.gz',
 '/Volumes/mrenkeED/data/ds-dnumr/derivatives/fmriprep/sub-01/ses-1/func/sub-01_ses-1_task-magjudge_run-2_space-T1w_desc-preproc_bold.nii.gz',
 '/Volumes/mrenkeED/data/ds-dnumr/derivatives/fmriprep/sub-01/ses-1/func/sub-01_ses-1_task-magjudge_run-3_space-T1w_desc-preproc_bold.nii.gz',
 '/Volumes/mrenkeED/data/ds-dnumr/derivatives/fmriprep/sub-01/ses-1/func/sub-01_ses-1_task-magjudge_run-4_space-T1w_desc-preproc_bold.nii.gz',
 '/Volumes/mrenkeED/data/ds-dnumr/derivatives/fmriprep/sub-01/ses-1/func/sub-01_ses-1_task-magjudge_run-5_space-T1w_desc-preproc_bold.nii.gz',
 '/Volumes/mrenkeED/data/ds-dnumr/derivatives/fmriprep/sub-01/ses-1/func/sub-01_ses-1_task-magjudge_run-6_space-T1w_desc-preproc_bold.nii.gz']

In [21]:
np.shape(onsets)

(366, 5)

In [22]:
366/6

61.0